In [7]:
import pandas as pd
import geopandas as gpd

In [8]:
noise_df = pd.read_csv("../00_data/01_raw/noise.csv")

In [9]:
noise_df

,GEOID,NAME,variable,estimate,moe,coverage_all,coverage_na,coverage_4050,coverage_5060,coverage_6070,...,noise6070n,noise7080n,noise8090n,noise90n,noise4050p,noise5060p,noise6070p,noise7080p,noise8090p,noise90p
0,1001020100,"Census Tract 201, Autauga County, Alabama",B01003_001,1941,390.0,1.097868e+04,1.010647e+04,315.702041,457.965519,94.755623,...,13.792972,0.534163,0.000000,0.000000,0.028756,0.041714,0.008631,0.000345,0.000000,0.000000e+00
1,1001020200,"Census Tract 202, Autauga County, Alabama",B01003_001,1757,310.0,3.697642e+03,3.277371e+03,155.037713,253.560340,11.673364,...,8.652346,0.000000,0.000000,0.000000,0.041929,0.068574,0.003157,0.000000,0.000000,0.000000e+00
2,1001020300,"Census Tract 203, Autauga County, Alabama",B01003_001,3694,570.0,5.975932e+03,5.360278e+03,223.309311,315.599470,75.533347,...,40.281346,0.578297,0.000000,0.000000,0.037368,0.052812,0.012640,0.000203,0.000000,0.000000e+00
3,1001020400,"Census Tract 204, Autauga County, Alabama",B01003_001,3539,500.0,7.072632e+03,6.193573e+03,252.459606,307.501209,313.098212,...,151.189170,4.229937,0.000000,0.000000,0.035695,0.043478,0.044269,0.000848,0.000000,0.000000e+00
4,1001020501,"Census Tract 205.01, Autauga County, Alabama",B01003_001,4306,662.0,6.910644e+03,5.831034e+03,328.280958,477.827056,219.875960,...,122.962396,22.420260,0.000000,0.000000,0.047504,0.069144,0.031817,0.007760,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83504,56043000200,"Census Tract 2, Washakie County, Wyoming",B01003_001,3124,198.0,6.433929e+06,6.416007e+06,7756.789861,7794.592064,2255.595731,...,5.021039,0.414610,0.180449,0.002683,0.001206,0.001211,0.000351,0.000013,0.000004,7.771301e-07
83505,56043000301,"Census Tract 3.01, Washakie County, Wyoming",B01003_001,2231,273.0,2.213791e+03,1.619494e+03,356.219819,206.161959,27.550139,...,24.265942,3.062596,0.000000,0.765652,0.160909,0.093126,0.012445,0.001578,0.000000,3.943781e-04
83506,56043000302,"Census Tract 3.02, Washakie County, Wyoming",B01003_001,2578,314.0,1.788184e+04,1.617065e+04,901.836298,612.464990,184.628494,...,19.380391,0.936672,0.106420,0.105612,0.050433,0.034251,0.010325,0.000559,0.000064,6.302088e-05
83507,56045951100,"Census Tract 9511, Weston County, Wyoming",B01003_001,3276,280.0,6.782478e+06,6.704607e+06,41550.185582,30418.644301,4925.389499,...,2.381369,0.376897,0.094266,0.023458,0.006126,0.004485,0.000726,0.000110,0.000027,7.519375e-06


In [10]:
# Extract the first 5 digits of 'GEOID' to match with 'fipsstatecode' + 'fipscountycode'
noise_df[['county', 'state']] = noise_df['NAME'].str.extract(r',\s*(.*? County),\s*(.*)')
noise_df['geoid_short'] = noise_df['GEOID'].astype(str).str[:5] 
noise_df_county = noise_df.groupby(by = ['geoid_short']).agg(
    {'coverage_5060':'sum','coverage_6070':'sum','coverage_7080':'sum','coverage_8090':'sum','coverage_90':'sum',
     'noise5060n':'sum','noise6070n':'sum','noise7080n':'sum','noise8090n':'sum','noise90n':'sum'}
).reset_index()

In [11]:
noise_df_county['coverage_50'] = noise_df_county['coverage_5060'] + noise_df_county['coverage_6070'] + noise_df_county['coverage_7080'] + noise_df_county['coverage_8090'] + noise_df_county['coverage_90']
noise_df_county['coverage_60'] = noise_df_county['coverage_6070'] + noise_df_county['coverage_7080'] + noise_df_county['coverage_8090'] + noise_df_county['coverage_90']
noise_df_county['coverage_70'] = noise_df_county['coverage_7080'] + noise_df_county['coverage_8090'] + noise_df_county['coverage_90']
noise_df_county['coverage_80'] = noise_df_county['coverage_8090'] + noise_df_county['coverage_90']

noise_df_county['noise50n'] = noise_df_county['noise5060n'] + noise_df_county['noise6070n'] + noise_df_county['noise7080n'] + noise_df_county['noise8090n'] + noise_df_county['noise90n']
noise_df_county['noise60n'] = noise_df_county['noise6070n'] + noise_df_county['noise7080n'] + noise_df_county['noise8090n'] + noise_df_county['noise90n']
noise_df_county['noise70n'] = noise_df_county['noise7080n'] + noise_df_county['noise8090n'] + noise_df_county['noise90n']
noise_df_county['noise80n'] = noise_df_county['noise8090n'] + noise_df_county['noise90n']


In [12]:
noise_df_county.to_csv('../00_data/02_intermediate/noise.csv',index=False)